In [1]:
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora       
from nltk.stem import WordNetLemmatizer as lemmatize

from gensim.utils import  simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
import numpy as np
import json, os, uuid
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/BarclaysReviews/data/final_merged_reviews.csv")


Mounted at /content/drive


In [3]:
MOUNT_DIR = '/content/drive/MyDrive/BarclaysReviews/'
DATA_DIR = '/content/drive/MyDrive/BarclaysReviews/data/'
MODEL_DIR = '/content/drive/MyDrive/BarclaysReviews/model/'
FILE_NAME ='Final_merged_positive_reviews.csv'

In [4]:
# barclays_stop_list = ['app', 'barclays','need','will','pay','work','let',"thing","able","unable","may","banking",'bank']
# replace_dict = {"switch":"switch apps","logininin":"login","loginin":"login","google pay":"googlepay","contactless card":"contactlesscard",
#                 "credit card":"credicard","debit card":"debitcard","calender view": "calender","push":"notification","time":"slow","log":"login",
#                 "mortgage account":"mortgage","poor":"bad","annoying":"bad","wait":"slow","push notification":"notification","frustrating":"bad","rubbish":"bad",
#                 "mobile":"device","long":"slow","find":"navigate","terrible":"slow","ask":"help","stop":"crash","keeps crashing":"crashing","cheques":"cheque"}


# great, slow, nan, especially, people, someslow, etc, "banking", "banking app","app", "bank", "don"
# {"excellent":"great,"brilliant":"great", "perfect":"great", "love": "great", "like": "great"}


barclays_stop_list = ["banking", "banking app","app", "bank", "don"]
replace_dict = {"love": "great", "like": "great"}

In [5]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy 


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:

nlp = spacy.load('en_core_web_sm')
nlp.Defaults.stop_words.update(barclays_stop_list)
nlp.Defaults.stop_words.update(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])
len(nlp.Defaults.stop_words)
%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [7]:
df = df[df['review_description'].notnull()]
df['review_description'] = df['review_description'].str.lower()
df['review_description'].replace(replace_dict,inplace=True,regex=True)
df['review_description'] = df['review_description'].str.lower()


<ipython-input-7-ea273063e480>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_description'] = df['review_description'].str.lower()


In [8]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# Convert to list
data = df.review_description.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

[['now', 'this', 'is', 'awful', 'because', 'cant', 'make', 'payments', 'with', 'my', 'phone', 'but', 'then', 'every', 'time', 'open', 'the', 'app', 'get', 'prompted', 'to', 'change', 'settings', 'so', 'that', 'can', 'make', 'payments', 'with', 'my', 'phone', 'and', 'after', 'ive', 'done', 'that', 'it', 'tells', 'me', 'that', 'cant', 'make', 'payments', 'with', 'my', 'phone', 'every', 'time', 'after', 'that', 'when', 'just', 'want', 'to', 'use', 'the', 'app', 'it', 'holds', 'everything', 'up', 'with', 'prompt', 'that', 'cant', 'make', 'payments', 'with', 'my', 'phone']]


In [9]:

from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('punkt')
import re
wnl = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [11]:
# Build the bigram and trigram models

bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # run in terminal once
def process_words(texts,allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in nlp.Defaults.stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    #nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    i =0
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        #print(doc)
        #texts_out.append([token.lemma_ for token in doc ])
         #[wnl.lemmatize(token.lemma_.lower()) for token in nlp(text) if not token.is_stop and not token.is_punct]
        regex_num_ponctuation = '(\d+)|([^\w\s])'
        regex_little_words = r'(\b\w{1,2}\b)'
        texts_out_temp = [token.lemma_ for token in doc if  not token.is_punct ]
        texts_out.append([wnl.lemmatize(t) for t in word_tokenize(' '.join(texts_out_temp)) 
              if not re.search(regex_num_ponctuation, t) and not re.search(regex_little_words, t)  ])
        print(texts_out[i])
        i = i+1
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in nlp.Defaults.stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)  # processed Text Data!

Streaming output truncated to the last 5000 lines.
['fast', 'safe', 'reliable']
['efficient', 'secure']
['excellent', 'xxx']
[]
[]
['quick', 'simple']
['insurmountable', 'issue', 'far', 'difficulty', 'pay', 'cheque', 'android', 'fail', 'successful', 'ipad']
['work', 'account', 'credit', 'card', 'statement', 'available', 'single', 'transaction', 'display', 'payment', 'amount', 'customer', 'service', 'call', 'multiple', 'time', 'absentia', 'help', 'fix', 'month', 'open', 'account', 'barclay', 'worth', 'will', 'not', 'able', 'actually', 'online', 'support']
['well', 'well']
['eye', 'account', 'minimum', 'time']
['excellent', 'service', 'keep', 'control', 'account']
['totally', 'useless', 'pay', 'cheque', 'cheque', 'flat', 'light', 'photo', 'cheque', 'come', 'weird', 'angle', 'see', 'great', 'not', 'understand', 'eventually', 'take', 'decent', 'photo', 'tell', 'picture', 'not', 'spend', 'half', 'hour', 'take', 'retake', 'photo', 'acceptable', 'useful', 'not', 'middle', 'pandemic']
['fast']

In [12]:

len(data_ready)
df['cleaned_data']=data_ready
df['cleaned_data']=df['cleaned_data'].apply(lambda x: ' '.join(x))
df = df[df['cleaned_data'].notnull()]
df.reset_index(drop=True, inplace=True)
def saveFileToDrive(path,dataframe):

  with open(path, 'w', encoding = 'utf-8-sig') as f:
    dataframe.to_csv(f)
    print("Saving file : "+path)


saveFileToDrive("/content/drive/MyDrive/BarclaysReviews/data/final_merged_reviews_cleaned.csv",df)

Saving file : /content/drive/MyDrive/BarclaysReviews/data/final_merged_reviews_cleaned.csv


In [13]:
pd.set_option('display.max_colwidth', -1)
df[df['source'].str.contains('Google Play') ].shape

<ipython-input-13-658365021a87>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


(30671, 18)

In [ ]:
df[df["review_description"].str.contains("amazon")][["review_description", "rating","cleaned_data"]].shape

(11, 3)